In [1]:
import os
import math
import random
import numpy as np
import tensorflow as tf
import cv2
import keyboard
import time
from pynput.mouse import Button, Controller
import wx
import playsound

In [2]:
slim = tf.contrib.slim
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import sys
sys.path.append('../')
from nets import ssd_vgg_300, ssd_common, np_methods
from preprocessing import ssd_vgg_preprocessing
from notebooks import visualization

In [3]:
app = wx.App(False)
mouse = Controller()

In [5]:
screen_x, screen_y = wx.GetDisplaySize()

In [6]:
# TensorFlow session: grow memory when needed. TF, DO NOT USE ALL MY GPU MEMORY!!!
gpu_options = tf.GPUOptions(allow_growth=True)
config = tf.ConfigProto(log_device_placement=False, gpu_options=gpu_options)
isess = tf.InteractiveSession(config=config)
# Input placeholder.
net_shape = (300, 300)
data_format = 'NHWC'
img_input = tf.placeholder(tf.uint8, shape=(None, None, 3))
# Evaluation pre-processing: resize to SSD net shape.
image_pre, labels_pre, bboxes_pre, bbox_img = ssd_vgg_preprocessing.preprocess_for_eval(
    img_input, None, None, net_shape, data_format, resize=ssd_vgg_preprocessing.Resize.WARP_RESIZE)
image_4d = tf.expand_dims(image_pre, 0)

In [7]:
# Define the SSD model.
reuse = True if 'ssd_net' in locals() else None
ssd_net = ssd_vgg_300.SSDNet()
with slim.arg_scope(ssd_net.arg_scope(data_format=data_format)):
    predictions, localisations, _, _ = ssd_net.net(image_4d, is_training=False, reuse=reuse)


In [8]:
# Restore SSD model.
#ckpt_filename = r'C:\Users\PRAVEEN\Desktop\Capstone_ssd\ssd_model_1\model.ckpt-25911'
ckpt_filename = r'C:\Users\PRAVEEN\Desktop\Capstone_ssd\SSD-Tensorflow-master\train_model_fine_tune_2\model.ckpt-38961'
# ckpt_filename = '../checkpoints/VGG_VOC0712_SSD_300x300_ft_iter_120000.ckpt'
isess.run(tf.global_variables_initializer())
saver = tf.train.Saver()
saver.restore(isess, ckpt_filename)
# SSD default anchor boxes.
ssd_anchors = ssd_net.anchors(net_shape)

INFO:tensorflow:Restoring parameters from C:\Users\PRAVEEN\Desktop\Capstone_ssd\SSD-Tensorflow-master\train_model_fine_tune_2\model.ckpt-38961


In [9]:
# Main image processing routine.
def process_image(img, select_threshold=0.2, nms_threshold=.8, net_shape=(300, 300)):
    # Run SSD network.
    rimg, rpredictions, rlocalisations, rbbox_img = isess.run([image_4d, predictions, localisations, bbox_img],
                                                              feed_dict={img_input: img})

    # Get classes and bboxes from the net outputs.
    rclasses, rscores, rbboxes = np_methods.ssd_bboxes_select(
        rpredictions, rlocalisations, ssd_anchors,
        select_threshold=select_threshold, img_shape=net_shape, num_classes=5, decode=True)

    rbboxes = np_methods.bboxes_clip(rbbox_img, rbboxes)
    rclasses, rscores, rbboxes = np_methods.bboxes_sort(rclasses, rscores, rbboxes, top_k=400)
    rclasses, rscores, rbboxes = np_methods.bboxes_nms(rclasses, rscores, rbboxes, nms_threshold=nms_threshold)
    # Resize bboxes to original image shape. Note: useless for Resize.WARP!
    rbboxes = np_methods.bboxes_resize(rbbox_img, rbboxes)
    return rclasses, rscores, rbboxes

In [17]:
cap = cv2.VideoCapture(0)
font = cv2.FONT_HERSHEY_SIMPLEX

#initialising defaults
pixel_threshold_1 = 30
pixel_threshold_2 = 40
pixel_threshold_3 = 40
thresh_val = 13.33

#count_l_dir = 0
start_pauser = 0
start_appch = 0
start_mouse = 0
start_l_mouse = 0
start_lc = 0

count_dir = 0
count_appch = 0
count_l_appch = 0
count_m = 0
#count_lc = 0

click_flag = 1
mouse_active = 1
# Define the codec and create VideoWriter object
#fourcc = cv2.VideoWriter_fourcc(*'XVID')
#out = cv2.VideoWriter('output.avi',fourcc, 20.0, (640,480))

while(True):
    #print(rclasses)
    # Capture frame-by-frame
    ret, img = cap.read()
    
    #if ret==True:
        #frame = cv2.flip(img,1)

        # write the flipped frame
        #out.write(frame)
    
    #info on height and width of each frame
    height = img.shape[0]
    width = img.shape[1]
    # Our operations on the frame come here
    #gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    rclasses, rscores, rbboxes =  process_image(img)
    
    if time.time() - start_pauser > 2:
        count_dir = 0
        count_l_dir = 0
    
    if time.time() - start_appch > 1.5:
        if count_appch > 0:
            keyboard.release('alt')
        count_appch = 0
        count_l_appch = 0
        
        
    if time.time() - start_l_mouse > 2:
        count_m = 0

        
    
    if rclasses.any() > 0:
        #print(rclasses)
        ymin = int(rbboxes[0, 0] * height)
        xmin = int(rbboxes[0, 1] * width)
        ymax = int(rbboxes[0, 2] * height)
        xmax = int(rbboxes[0, 3] * width)
        # Display the resulting frame
        cv2.putText(img, (str(rclasses[0]) + "-" + str(rscores[0])),(xmax, ymax), font, 2, (26,255,255))
        #cv2.putText(img, str(rscores[0]),(int((xmax+xmin)/2), ymax), font, 2, (255,255,255))
        cv2.rectangle(img, (xmin, ymin), (xmax, ymax), (0, 255, 0), 2)
        cv2.imshow('frame',img)
        count_dir += 1
        count_appch += 1
        count_m += 1
        #dynamic thresholding
        area = ((xmin - xmax)*(ymin - ymax))/100
        pixel_threshold_1 = pixel_threshold_2 = pixel_threshold_3 = area/thresh_val
        print(pixel_threshold_1)
        #keyboard controls for up, down, left, right, space
        if (rclasses[0] in [4,5]) and mouse_active > 0:
            x_mid = (xmin + xmax)/2
            y_mid = (ymin + ymax)/2
            if count_dir == 1:
                start_pauser = time.time()
                x_mid_1 = x_mid
                y_mid_1 = y_mid
            elif ((count_dir > 1) and ((time.time() - start_pauser) > 0.5) and (x_mid < (x_mid_1 + pixel_threshold_1)) and (x_mid > (x_mid_1 - pixel_threshold_1)) and (y_mid < (y_mid_1 + pixel_threshold_1)) and (y_mid > (y_mid_1 - pixel_threshold_1))):
                keyboard.press_and_release('space')
                #print("space")
                time.sleep(0.75)
                #count = 0
            elif ((count_dir > 1) and (x_mid > (x_mid_1 + pixel_threshold_2)) and ((time.time() - start_pauser) > 0.15)):
                keyboard.press_and_release('left arrow')
                #print("left arrow")
                #if count_l > 3:
                #    keyboard.press('left arrow')
                #time.sleep(0.25)
                #count_l += 1
                start_pauser = time.time()
            elif ((count_dir > 1) and (x_mid < (x_mid_1 - pixel_threshold_2)) and ((time.time() - start_pauser) > 0.15)):
                keyboard.press_and_release('right arrow')
                #print("right arrow")
                #time.sleep(0.75)
                #count_l += 1
                start_pauser = time.time()
            elif ((count_dir > 1) and (y_mid > (y_mid_1 + pixel_threshold_3)) and ((time.time() - start_pauser) > 0.15)):
                #print('yyyyyyy')
                keyboard.press_and_release('down arrow')
                #time.sleep(0.75)
                #count_l += 1
                start_pauser = time.time()
            elif ((count_dir > 1) and (y_mid < (y_mid_1 - pixel_threshold_3)) and ((time.time() - start_pauser) > 0.15)):
                #print('nnnnnnnnnn')
                keyboard.press_and_release('up arrow')
                #time.sleep(0.75)
                #count_l += 1
                start_pauser = time.time()
                
        #task switcher function        
        if (rclasses[0] in [3]) and mouse_active > 0:
            x_mid = (xmin + xmax)/2
            y_mid = (ymin + ymax)/2
            if count_appch == 1:
                start_appch = time.time()
            elif count_appch > 1 and (time.time() - start_appch) > 0.5 and count_l_appch == 0:
                x_mid_1 = x_mid
                y_mid_1 = y_mid
                keyboard.press('alt')
                count_l_appch += 1
            elif ((count_appch > 1) and (count_l_appch > 0) and ((time.time() - start_appch) > 0.15) and (x_mid < (x_mid_1 + pixel_threshold_1)) and (x_mid > (x_mid_1 - pixel_threshold_1)) and (y_mid < (y_mid_1 + pixel_threshold_1)) and (y_mid > (y_mid_1 - pixel_threshold_1))):
                keyboard.press_and_release('tab')
                time.sleep(0.75)
                start_appch = time.time()
        
        #switch between mouse and keyboard
        if rclasses[0] in [1]:
            if count_m == 1:
                start_mouse = time.time()
                start_l_mouse = time.time()
            elif count_m > 1:
                start_l_mouse = time.time()
            if (time.time() - start_mouse) > 2:
                mouse_active = mouse_active * (-1)
                if mouse_active < 0:
                    playsound.playsound('mouse activated.mp3')
                elif mouse_active > 0:
                    playsound.playsound('mouse deactivated.mp3')
                time.sleep(2)
                
        #mouse functions
        if mouse_active < 0:
            if rclasses[0] in [4]:
                #x_mid = (xmin + xmax)/2
                #y_mid = (ymin + ymax)/2
                #mouse.position = (screen_x -  (xmin * screen_x/(width - 150)), ymin * screen_y/(height - 200))
                x_mid = (width/2 - (xmin + xmax)/2)/10
                y_mid = ((ymin + ymax)/2 - height/2)/10
                #if x_mid > 0 :
                #    x_mid = 5
                #else:
                #    x_mid = -5
                #if y_mid > 0:
                #    y_mid = 5
                #else:
                #    y_mid = -5
                mouse.move(x_mid, y_mid)
                
            if rclasses[0] in [2]:
                if click_flag > 0:
                    mouse.press(Button.left)
                    click_flag = click_flag * (-1)
                start_lc = time.time()
                x_mid = (width/2 - (xmin + xmax)/2)/10
                y_mid = ((ymin + ymax)/2 - height/2)/10
                mouse.move(x_mid, y_mid)
                #count_lc += 1
            if (time.time() - start_lc) > 0.25 and click_flag < 0:
                mouse.release(Button.left)
                click_flag = 1
                #count_lc = 0
                
            if rclasses[0] in [5]:
                mouse.click(Button.left, 2)
                time.sleep(0.5)
     
        #if rclasses[0] in [1,2,3]:    
    
    cv2.imshow('frame',img)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the capture
cap.release()
#out.release()
cv2.destroyAllWindows()

32.30007501875469
31.909227306826708
31.69092273068267
31.953488372093023
32.29707426856714
27.124531132783193
32.4688672168042
32.66916729182296
32.01500375093774
31.984996249062267
32.3953488372093
33.00825206301575
33.458364591147784
33.79219804951238
33.4748687171793
32.87171792948237
33.33833458364591
33.97449362340585
32.32708177044261
33.44561140285071
33.21080270067517
31.75993998499625
31.514628657164288
29.111027756939237
33.54838709677419
34.34358589647412
34.1635408852213
34.34358589647412
34.02325581395348
34.829707426856714
34.55063765941485
34.18604651162791
34.18604651162791
35.15378844711178
30.148537134283572
31.87396849212303
34.893473368342086
36.17104276069018
35.98649662415604
38.25956489122281
35.94748687171793
33.47336834208552
32.19804951237809
35.53038259564891
35.30832708177044
34.18679669917479
33.78094523630908
33.764441110277566
34.700675168792195
35.50037509377344
37.52438109527382
35.32108027006752
36.70142535633909
38.072018004501125
36.942985746436605


In [160]:
cap.release()
cv2.destroyAllWindows()